In [1]:
!pip install -U deepctr-torch

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: deepctr-torch in /home/v-tyan/.local/lib/python3.7/site-packages (0.2.4)


In [2]:
import pandas as pd
import numpy as np 
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import DeepFM
import pandas as pd
import os
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from io import StringIO
import re 
import pickle
import scipy.io.arff
from torch.optim import Adam

dataset_path = "/home/v-tyan/NN_for_tabular/datasets_raw/"


In [4]:
df = pd.read_csv(os.path.join(dataset_path, "appetency_churn_upselling/orange_small_train.data"), sep = "\t")
cols = list(df.columns)
cat_cols = [cols[idx] for idx in [190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206,
            207, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228]]
num_cols = list(set(cols) - set(cat_cols))
dataset_ = 'appetency'
X, Y = df, -pd.read_csv(os.path.join(dataset_path, f'appetency_churn_upselling/orange_small_train_{dataset_}.labels'), header=None)[0]
# train_idx = pd.read_csv(os.path.join(dataset_path, f'appetency_churn_upselling/{dataset_}/stratified_train_idx_{dataset_}.txt'), header=None)
# test_idx = pd.read_csv(os.path.join(dataset_path, f'appetency_churn_upselling/{dataset_}/stratified_test_idx_{dataset_}.txt'), header=None)
# train_X, test_X, train_Y, test_Y = X.iloc[train_idx[0]], X.iloc[test_idx[0]], Y.iloc[train_idx[0]], Y.iloc[test_idx[0]]
columns = X.columns
data = pd.concat((X, Y), axis=1)
data

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,0
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN,1
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,NaN,NaN,NaN,NaN,NaN,357.0,0.0,NaN,NaN,NaN,...,EROH7Cg,LM8l689qOp,NaN,NaN,7FJQ,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
49996,NaN,NaN,NaN,NaN,NaN,1078.0,0.0,NaN,NaN,NaN,...,GfSQowC,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,am7c,NaN,-1
49997,NaN,NaN,NaN,NaN,NaN,2807.0,7.0,NaN,NaN,NaN,...,dh6qI2t,LM8l689qOp,NaN,ELof,fKCe,RAYp,TCU50_Yjmm6GIBZ0lL_,NaN,NaN,1
49998,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2fF2Oqu,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,1


In [10]:
sparse_features = cat_cols
dense_features = num_cols
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = [0]

In [11]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat].apply(str))
# mms = MinMaxScaler(feature_range=(0,1))
# data[dense_features] = mms.fit_transform(data[dense_features])

In [12]:
data[sparse_features]

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0,142,18,0,15,0,88,2561,3354,0,...,4225,4,2184,4,0,0,15,2,28,0
1,0,354,18,0,15,0,46,2998,4325,0,...,2,4,280,1,0,0,6,2,28,0
2,0,167,45,2,15,0,66,764,4337,5031,...,328,0,681,4,0,2,2,0,18,1
3,0,191,18,0,15,0,117,1752,2924,0,...,496,4,4089,1,0,0,20,2,28,0
4,0,38,18,2,15,0,123,4260,1559,7085,...,815,4,3903,1,0,2,20,2,28,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,271,18,0,15,0,45,3804,2784,0,...,1746,4,4257,1,0,0,16,2,28,0
49996,0,91,0,2,21,0,198,2523,4627,4574,...,572,4,142,1,0,2,20,2,12,1
49997,0,254,0,0,15,0,149,548,3009,14266,...,49,4,2040,1,0,1,6,2,6,0
49998,1,20,18,0,15,0,155,1377,859,0,...,4111,4,469,1,0,0,20,2,28,0


In [13]:
sparse_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4) for i,feat in enumerate(sparse_features)]
dense_feature_columns = [DenseFeat(feat, 1,) for feat in dense_features]
dnn_feature_columns = sparse_feature_columns + sparse_feature_columns
linear_feature_columns = sparse_feature_columns + dense_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [14]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name] for name in feature_names}

test_model_input = {name:test[name] for name in feature_names}

for x in train_model_input:
    print(train_model_input[x].shape)


device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(linear_feature_columns,dnn_feature_columns, init_std=0.1,task='binary',device=device)
model.compile(Adam(model.parameters(), 5e-4), "binary_crossentropy",
              metrics=['binary_crossentropy'], )

history = model.fit(train_model_input,lbe.fit_transform(train[target].values),epochs=30,verbose=2,validation_split=0.25)
pred_ans = model.predict(test_model_input, batch_size=256)
print(f'{log_loss(test[target].values, pred_ans):.5f}')

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(40000,)
(

ValueError: y_true contains only one label (1.0). Please provide the true labels explicitly through the labels argument.